<img src="../docs/choice_learn_official_logo.png" width="256">

Choice-Learn is a Python package designed to help building discrete choice models. In particular you will find:

- Optimized **Data** handling with the ChoiceDataset object and ready-to-use datasets
- **Modelling** tools with:
    - Efficient well-known choice models
    - Customizable class ChoiceModel to build your own model
    - Estimation options such as choosing the method (LBFGS, Gradient Descent, etc...)
- Divers **Tools** revolving around choice models such as an Assortment Optimizer


### Discrete Choice Modelling
Discrete choice models aim at explaining or predicting a choice from a set of alternatives. Well known use-cases include analyzing people choice of mean of transport or products purchases in stores.

If you are new to choice modelling, you can check this [resource](https://www.publichealth.columbia.edu/research/population-health-methods/discrete-choice-model-and-analysis). 

### Tutorial
In this notebook we will describe step-by-step the estimation of a choice model.


